# **Get Match Data + Odds Data**

## Intro

### Imports

In [1]:
from sportmonks.soccer import SoccerApiV2
from datetime import date
import pandas as pd
import collections
import time 
import pandas as pd
import json

soccer = SoccerApiV2(api_token='EJnOPHRD4WyO4W4np63izq5Fw7RuwLu2Ys0DiUWy6e4BdV5DUQNmf8uPIkvr')
soccer.meta()

{'subscription': {'started_at': {'date': '2022-04-12 13:21:13.000000',
   'timezone_type': 3,
   'timezone': 'UTC'},
  'trial_ends_at': {'date': '2022-04-26 14:18:45.000000',
   'timezone_type': 3,
   'timezone': 'UTC'},
  'ends_at': {'date': '2022-05-12 13:21:09.000000',
   'timezone_type': 3,
   'timezone': 'UTC'}},
 'plans': [{'name': 'European Plan',
   'features': 'Advanced',
   'price_monthly': 57,
   'price_yearly': 600,
   'request_limit': '2000,60',
   'sport': 'Soccer'}],
 'add-ons': [],
 'sports': [{'id': 1, 'name': 'Soccer', 'current': True}]}

In [2]:
soccer.http_requests_made # Number of HTTP requests made

0

In [3]:
# soccer.seasons()
soccer.leagues()

[{'id': 2,
  'active': True,
  'type': 'cup_international',
  'legacy_id': 11,
  'country_id': 41,
  'logo_path': 'https://cdn.sportmonks.com/images/soccer/leagues/2.png',
  'name': 'Champions League',
  'is_cup': True,
  'is_friendly': False,
  'current_season_id': 18346,
  'current_round_id': None,
  'current_stage_id': 77453619,
  'live_standings': True,
  'coverage': {'predictions': True,
   'topscorer_goals': True,
   'topscorer_assists': True,
   'topscorer_cards': True}},
 {'id': 5,
  'active': True,
  'type': 'cup_international',
  'legacy_id': 12,
  'country_id': 41,
  'logo_path': 'https://cdn.sportmonks.com/images/soccer/leagues/5.png',
  'name': 'Europa League',
  'is_cup': True,
  'is_friendly': False,
  'current_season_id': 18629,
  'current_round_id': None,
  'current_stage_id': 77454496,
  'live_standings': True,
  'coverage': {'predictions': True,
   'topscorer_goals': True,
   'topscorer_assists': True,
   'topscorer_cards': True}},
 {'id': 8,
  'active': True,
  'typ

### Get all fixtures for each league

In [4]:
today = '2022-04-22'
cups_fix_dict = dict()
leagues_fix_dict = dict()

In [5]:
# CHAMPIONS LEAGUE 
cups_fix_dict['cl_fixtures'] = soccer.fixtures(start_date = '2011-08-01', end_date = today, league_ids = 2) 
# EUROPA LEAGUE
cups_fix_dict['el_fixtures'] = soccer.fixtures(start_date = '2011-08-01', end_date = today, league_ids = 5)
# PREMIER LEAGUE 
leagues_fix_dict['pl_fixtures'] = soccer.fixtures(start_date = '2011-08-01', end_date = today, league_ids = 8)
# FA CUP
cups_fix_dict['fa_fixtures'] = soccer.fixtures(start_date = '2011-08-01', end_date = today, league_ids = 24)
# BUNDESLIGA 
leagues_fix_dict['bu_fixtures'] = soccer.fixtures(start_date = '2011-08-01', end_date = today, league_ids = 82)
# LIGUE 1 
leagues_fix_dict['l1_fixtures'] = soccer.fixtures(start_date = '2011-08-01', end_date = today, league_ids = 301)
# SERIE A 
leagues_fix_dict['sa_fixtures'] = soccer.fixtures(start_date = '2011-08-01', end_date = today, league_ids = 384)
# COPPA ITALIA
cups_fix_dict['ci_fixtures'] = soccer.fixtures(start_date = '2011-08-01', end_date = today, league_ids = 390)
# LA LIGA
leagues_fix_dict['ll_fixtures'] = soccer.fixtures(start_date = '2011-08-01', end_date = today, league_ids = 564)
# COPA DEL REY 
cups_fix_dict['dr_fixtures'] = soccer.fixtures(start_date = '2011-08-01', end_date = today, league_ids = 570)

In [6]:
cups_all_fixtures = []
leagues_all_fixtures = []

for cupvalues in cups_fix_dict.values():
    for fix in cupvalues:
        cups_all_fixtures.append(fix['id'])
        
for leaguevalues in leagues_fix_dict.values():
    for fix in leaguevalues:
        leagues_all_fixtures.append(fix['id'])

In [7]:
print(len(cups_all_fixtures))
print(len(leagues_all_fixtures))

18309
19838


## For each fixture get important Data 

In [24]:
# Turn a nested dictionary into a flattened dictionary
def flatten(d, parent_key='', sep='_'): 
    items = []
    for k, v in d.items():
        new_key = parent_key + sep + k if parent_key else k
        if isinstance(v, collections.MutableMapping):
            items.extend(flatten(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)

# Return a list of all fixtures in fix_ids_list controlling for the API calls limit
def list_all_fixtures(fix_ids_list):
    complete = []
    for index, i in enumerate(fix_ids_list):
        try:
            complete.append(flatten(soccer.fixture(fixture_id=i, includes=['localTeam', 'visitorTeam', 'referee', 'localCoach', 'visitorCoach', 'round', 'stats', 'league', 'season', 'venue'])))
        except Exception as e:
            print('************** An exception occurred: {} **************'.format(e))
            print('************** At Index: {} **************'.format(index))
            time.sleep(10)
        
        if (index + 1) % 1980 == 0: 
            print('STOP N°: ', int((index+1) / 1980),' -------- PERC: ', round((index+1)/len(fix_ids_list)*100, 2), '%')
            print('!!! WAIT ONE HOUR !!!')
            time.sleep(3600)
    return complete 

In [ ]:
cups_complete = list_all_fixtures(cups_all_fixtures)

In [ ]:
time.sleep(3600)
leagues_complete = list_all_fixtures(leagues_all_fixtures)

In [ ]:
print(len(cups_complete) == len(cups_all_fixtures))
print(len(leagues_complete) == len(leagues_all_fixtures))

In [ ]:
cups = pd.DataFrame(cups_complete)
cups.to_csv("Input Data/cups_static.csv", encoding='utf-8', index=False)

leagues = pd.DataFrame(leagues_complete)
leagues.to_csv("Input Data/leagues_static.csv", encoding='utf-8', index=False)

## For each fixture get Odds Data 

#### Functions

In [8]:
abbr1 = {'3WayResult': '3W', 'AsianHandicap' : 'AH', 'GoalsOver/Under': 'O/U', 'CorrectScore' : 'CS'}
abbr2 = {'Home' : 'H', 'Away' : 'A', 'Over' : 'O', 'Under' : 'U', 'Handicap' : 'Hnd', 'Result': 'RES', 'TotalGoals': 'TG'}

def myReplace(text):
    global abbr1, abbr2
    for key in abbr1:
        text = text.replace(key, abbr1[key])
    for key in abbr2:
        text = text.replace(key, abbr2[key])
    return text

In [9]:
def odds_ordered(unordered_data):
    bets_to_exclude = ['Goalscorer', 'Multi Scorers', 'Team Goalscorer', 'Player to be Booked', 'Player to be Sent Off']
    final_list = []
    for fix in range(len(unordered_data)):
        d = {}
        d['id'] = unordered_data[fix]['match_id'] # get match id
        for i in range(len(unordered_data[fix]['odds_infos'])): # loop only in odds_infos value
            if unordered_data[fix]['odds_infos'][i]['name'] not in bets_to_exclude: # excluding some bets' types
                odd_name = unordered_data[fix]['odds_infos'][i]['name'].replace(' ', '') # get odds_type
            for j in range(len(unordered_data[fix]['odds_infos'][i]['bookmaker'])): # loop for each bookmaker 
                bookmaker_name = unordered_data[fix]['odds_infos'][i]['bookmaker'][j]['name'].replace(' ', '') # get bookmaker name
                for k in range(len(unordered_data[fix]['odds_infos'][i]['bookmaker'][j]['odds'])): # loop for each possible bet
                    last_dict = unordered_data[fix]['odds_infos'][i]['bookmaker'][j]['odds'][k] # create a new dictionary with only important values 
                    label = last_dict['label'] # label and value are the two fundamental values 
                    value = last_dict['value']
                    if 'total' in last_dict.values(): # consider total, handicap and extra only if they exist
                        total = last_dict['total']
                    else:
                        total = None
                    if 'handicap' in last_dict.values():
                        handicap = last_dict['handicap']
                    else:
                        handicap = None
                    if 'extra' in last_dict.values():
                        extra = last_dict['extra']
                    else:
                        extra = None
                    book_info = odd_name + '_' + bookmaker_name + '__' # create keys' names combining all the bet main characteristics 
                    book_info = myReplace(book_info)
                    if total is None and handicap is None and extra is None: # filling the dictionary 
                        d[book_info + str(label)] = value
                    elif total is not None and handicap is None and extra is None:
                        d[book_info + str(label) + '_' + str(total)] = value
                    elif total is None and handicap is not None and extra is None:
                        d[book_info + str(label) + '_' + str(handicap)] = value
                    elif total is None and handicap is  None and extra is not None:
                        d[book_info + str(label) + '_' + str(extra)] = value
                    else:
                        d[book_info + str(label) + '_' + str(handicap) + '_' + str(total) + '_' + str(extra)] = value
        final_list.append(d)
    return final_list

In [10]:
def list_all_odds(fix_ids_list):
    complete = []
    for index, i in enumerate(fix_ids_list):
        d = {}
        d['match_id'] = i
        try:
            d['odds_infos'] = soccer.pre_match_odds(fixture_id=i)
        except Exception as e:
            print('************** An exception occurred: {} **************'.format(e))
            print('************** At Index: {} **************'.format(index))
            d['exception'] = index
            time.sleep(10)
        finally:
            complete.append(d)
        if (index + 1) % 1999 == 0: 
            print('STOP N°: ', int((index+1) / 1999),' -------- PERC: ', round((index+1)/len(fix_ids_list)*100, 2), '%')
            print('!!! WAIT ONE HOUR !!!')
            time.sleep(3600)
    return complete 

#### Leagues Data

In [ ]:
time.sleep(3600)
leagues_complete_odds = list_all_odds(leagues_all_fixtures)

In [ ]:
# LEAGUES
# Delete exceptions raised in previous cell
leagues_complete_odds[12814].pop('exception', None)
leagues_complete_odds[12817].pop('exception', None)
# Add the correct info 
leagues_complete_odds[12814]['odds_infos'] = soccer.pre_match_odds(fixture_id=299235)
leagues_complete_odds[12817]['odds_infos'] = soccer.pre_match_odds(fixture_id=299273)

In [ ]:
# CHECK FOR THE RIGHT KEYS 
for i in range(len(leagues_complete_odds)): # for leagues
    for k in leagues_complete_odds[i].keys():
        if k not in ['match_id', 'odds_infos']:
            print(leagues_complete_odds[i])

In [ ]:
final_leagues_odds = odds_ordered(leagues_complete_odds)    

if len(final_leagues_odds) == len(leagues_complete_odds):
    with open('Input Data/final_leagues_odds.json', 'w', encoding='utf-8') as f:
        json.dump(final_leagues_odds, f, ensure_ascii=False, indent=4)
    f.close()

#### Cups Data

In [13]:
cups_complete_odds = list_all_odds(cups_all_fixtures) 

************** An exception occurred: HTTPSConnectionPool(host='soccer.sportmonks.com', port=443): Max retries exceeded with url: /api/v2.0/odds/fixture/1061954?api_token=EJnOPHRD4WyO4W4np63izq5Fw7RuwLu2Ys0DiUWy6e4BdV5DUQNmf8uPIkvr&tz=Europe%2FRome&include=&page=1 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7fbd9a5cf490>: Failed to establish a new connection: [Errno 60] Operation timed out')) **************
************** At Index: 1054 **************
STOP N°:  1  -------- PERC:  10.92 %
!!! WAIT ONE HOUR !!!
STOP N°:  2  -------- PERC:  21.84 %
!!! WAIT ONE HOUR !!!
STOP N°:  3  -------- PERC:  32.75 %
!!! WAIT ONE HOUR !!!
STOP N°:  4  -------- PERC:  43.67 %
!!! WAIT ONE HOUR !!!
STOP N°:  5  -------- PERC:  54.59 %
!!! WAIT ONE HOUR !!!
STOP N°:  6  -------- PERC:  65.51 %
!!! WAIT ONE HOUR !!!
STOP N°:  7  -------- PERC:  76.43 %
!!! WAIT ONE HOUR !!!
STOP N°:  8  -------- PERC:  87.35 %
!!! WAIT ONE HOUR !!!
************** An exception occurred:

In [15]:
print(cups_complete_odds[1054])
print(cups_complete_odds[16250])
print(cups_complete_odds[16252])
print(cups_complete_odds[16261])

{'match_id': 1061954, 'exception': 1054}
{'match_id': 1087354, 'exception': 16250}
{'match_id': 1087375, 'exception': 16252}
{'match_id': 1086772, 'exception': 16261}


In [17]:
# CUPS
# Delete exceptions raised in previous cell
cups_complete_odds[1054].pop('exception', None)
cups_complete_odds[16250].pop('exception', None)
cups_complete_odds[16252].pop('exception', None)
cups_complete_odds[16261].pop('exception', None)
# Add the correct info 
cups_complete_odds[1054]['odds_infos'] = soccer.pre_match_odds(fixture_id=1061954)
cups_complete_odds[16250]['odds_infos'] = soccer.pre_match_odds(fixture_id=1087354)
cups_complete_odds[16252]['odds_infos'] = soccer.pre_match_odds(fixture_id=1087375)
cups_complete_odds[16261]['odds_infos'] = soccer.pre_match_odds(fixture_id=1086772)

In [18]:
# CHECK FOR THE RIGHT KEYS 
for i in range(len(cups_complete_odds)): # for cups
    for k in cups_complete_odds[i].keys():
        if k not in ['match_id', 'odds_infos']:
            print(cups_complete_odds[i])

In [19]:
final_cups_odds = odds_ordered(cups_complete_odds)     

if len(final_cups_odds) == len(cups_complete_odds):
    with open('Input Data/final_cups_odds.json', 'w', encoding='utf-8') as f:
        json.dump(final_cups_odds, f, ensure_ascii=False, indent=4)
    f.close()

## More Features Data

#### Functions

In [25]:
# Return a list of all fixtures in fix_ids_list controlling for the API calls limit
def list_more(fix_ids_list):
    complete = []
    for index, i in enumerate(fix_ids_list):
        try:
            complete.append(flatten(soccer.fixture(fixture_id=i, includes=['substitutions', 'goals', 'cards', 'corners', 'lineup', 'bench', 'sidelined'])))
        except Exception as e:
            print('************** An exception occurred: {} **************'.format(e))
            print('************** At Index: {} **************'.format(index))
            time.sleep(10)
        
        if (index + 1) % 1999 == 0: 
            print('STOP N°: ', int((index+1) / 1999),' -------- PERC: ', round((index+1)/len(fix_ids_list)*100, 2), '%')
            print('!!! WAIT ONE HOUR !!!')
            time.sleep(3600)
            print('--- START MAKING API CALLS ---')
    return complete 

#### Leagues More

In [26]:
time.sleep(3600)
leagues_more = list_more(leagues_all_fixtures)

/var/folders/xr/y1bkzkl93sl5hxsn03468vzh0000gn/T/ipykernel_13496/1782479099.py:6: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.10 it will stop working
  if isinstance(v, collections.MutableMapping):


STOP N°:  1  -------- PERC:  10.08 %
!!! WAIT ONE HOUR !!!
--- START MAKING API CALLS ---
STOP N°:  2  -------- PERC:  20.15 %
!!! WAIT ONE HOUR !!!
--- START MAKING API CALLS ---
STOP N°:  3  -------- PERC:  30.23 %
!!! WAIT ONE HOUR !!!
--- START MAKING API CALLS ---
STOP N°:  4  -------- PERC:  40.31 %
!!! WAIT ONE HOUR !!!
--- START MAKING API CALLS ---
STOP N°:  5  -------- PERC:  50.38 %
!!! WAIT ONE HOUR !!!
--- START MAKING API CALLS ---
STOP N°:  6  -------- PERC:  60.46 %
!!! WAIT ONE HOUR !!!
--- START MAKING API CALLS ---
************** An exception occurred: HTTPSConnectionPool(host='soccer.sportmonks.com', port=443): Max retries exceeded with url: /api/v2.0/fixtures/302049?api_token=EJnOPHRD4WyO4W4np63izq5Fw7RuwLu2Ys0DiUWy6e4BdV5DUQNmf8uPIkvr&tz=Europe%2FRome&include=bench%2Ccards%2Ccorners%2Cgoals%2Clineup%2Csidelined%2Csubstitutions&page=1 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7fbd98bfcac0>: Failed to establish a new connection: 

In [35]:
leagues_more.append(flatten(soccer.fixture(fixture_id=leagues_all_fixtures[13763], includes=['substitutions', 'goals', 'cards', 'corners', 'lineup', 'bench', 'sidelined'])))
leagues_more.append(flatten(soccer.fixture(fixture_id=leagues_all_fixtures[13772], includes=['substitutions', 'goals', 'cards', 'corners', 'lineup', 'bench', 'sidelined'])))

In [36]:
print(len(leagues_more))
print(len(leagues_all_fixtures))

19838
19838


In [37]:
leagues = pd.DataFrame(leagues_more)
leagues.to_csv("Input Data/leagues_more.csv", encoding='utf-8', index=False)

#### Cups More

In [27]:
time.sleep(3600)
cups_more = list_more(cups_all_fixtures)

STOP N°:  1  -------- PERC:  10.92 %
!!! WAIT ONE HOUR !!!
--- START MAKING API CALLS ---
STOP N°:  2  -------- PERC:  21.84 %
!!! WAIT ONE HOUR !!!
--- START MAKING API CALLS ---
STOP N°:  3  -------- PERC:  32.75 %
!!! WAIT ONE HOUR !!!
--- START MAKING API CALLS ---
STOP N°:  4  -------- PERC:  43.67 %
!!! WAIT ONE HOUR !!!
--- START MAKING API CALLS ---
STOP N°:  5  -------- PERC:  54.59 %
!!! WAIT ONE HOUR !!!
--- START MAKING API CALLS ---
************** An exception occurred: HTTPSConnectionPool(host='soccer.sportmonks.com', port=443): Max retries exceeded with url: /api/v2.0/fixtures/1070867?api_token=EJnOPHRD4WyO4W4np63izq5Fw7RuwLu2Ys0DiUWy6e4BdV5DUQNmf8uPIkvr&tz=Europe%2FRome&include=bench%2Ccards%2Ccorners%2Cgoals%2Clineup%2Csidelined%2Csubstitutions&page=1 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7fbd99659bb0>: Failed to establish a new connection: [Errno 60] Operation timed out')) **************
************** At Index: 9997 **********

In [38]:
cups_more.append(flatten(soccer.fixture(fixture_id=cups_all_fixtures[9997], includes=['substitutions', 'goals', 'cards', 'corners', 'lineup', 'bench', 'sidelined'])))

In [39]:
print(len(cups_more))
print(len(cups_all_fixtures))

18309
18309


In [40]:
cups = pd.DataFrame(cups_more)
cups.to_csv("Input Data/cups_more.csv", encoding='utf-8', index=False)